In [1]:
# libraries
import math
import numpy as np
import pandas as pd
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from datetime import date

In [2]:
objects = ['AAPL',
           'CL=F']

In [3]:
df = web.DataReader(objects[1], data_source='yahoo', start='2012-01-01', end=date.today())

In [ ]:
# visualization of closing price history
plt.figure(figsize=(16,8))
plt.title('Closing Price History')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.show()

In [4]:
data = df.filter(['Close'])
dataset = data.values
training_len = math.ceil(len(dataset) * 0.8)

In [5]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

In [6]:
periods = [5,10,20,30]
period = 5

In [ ]:
training_data = scaled_data[ : training_len , : ]
x_train = []
y_train = []
for i in range(period,len(training_data)):
  x_train.append(training_data[i-period : i , 0])
  y_train.append(training_data[i,0])
x_train,y_train = np.array(x_train),np.array(y_train)
print(x_train.shape)
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
print(x_train.shape)

In [ ]:
print(y_train.shape)

In [ ]:
layers = [1,2,3,4]
neurons = [10,20,30,40,50,60]
neurons_num = 60
dropouts = [0.0,0.1,0.2,0.3,0.4,0.5]
dropout = dropouts[5]

In [ ]:
# model definition.
model = Sequential()
model.add(LSTM(neurons_num, input_shape=(x_train.shape[1],1)))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
testing_data = scaled_data[training_len - period : , : ]
x_test = []
y_test = dataset[training_len :, :]
for i in range(period,len(testing_data)):
  x_test.append(testing_data[i-period : i , 0])
x_test = np.array(x_test)
print(x_test.shape)
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
print(x_test.shape)

In [7]:
def make_train_test(train_len, test_len, data):
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    for i in range(train_len):
        x_train.append(data[i:i+period, 0])
        y_train.append(data[i+period, 0])
    for j in range(test_len):
        x_test.append(data[j+train_len:j+train_len+period, 0])
        y_test.append(data[j+train_len+period, 0])
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_train = np.reshape(x_train ,(x_train.shape[0],x_train.shape[1],1))
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
    return x_train, y_train, x_test, y_test

In [13]:
folds = 6
unit_len = math.ceil(len(data)/(folds+1))

In [12]:
print(y_test[:10])

[0.90449513 0.89254861 0.89261612 0.88417929 0.89383102 0.88978131
 0.8924811  0.89612581 0.88195196 0.88762148]


In [ ]:
# predict.
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
train = data[ : training_len]
valid = data[training_len : ]
valid['Predictions'] = predictions

In [16]:
RMSDs = []
for i in range(folds):
    x_train, y_train, x_test, y_test = make_train_test(unit_len*(i+1), unit_len, scaled_data)
    model.fit(x_train,y_train,batch_size=1,epochs=15)
    predictions = model.predict(x_test)
    rmsd = np.sqrt(np.mean(predictions - y_test)**2)
    RMSDs.append(rmsd)
np.mean(RMSDs)

NameError: name 'model' is not defined

In [ ]:
print(predictions)

In [ ]:
# plot.
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()
# root-mean-square deviation.
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse